# 🧠 Sumarização de Texto com LLMs — Execução Mínima

Este roteiro mostra como aplicar um modelo LLM pré-treinado para gerar resumos de textos em português, usando a biblioteca 🤗 Hugging Face.

---

# 📥 Carregar modelo T5 adaptado para português

In [ ]:
from transformers import pipeline

# Modelo adaptado para sumarização em português
modelo = "unicamp-dl/ptt5-base-portuguese-vocab"

# Pipeline de sumarização
sumarizador = pipeline("summarization", model=modelo, tokenizer=modelo)


# 📝 Texto de entrada

In [ ]:
texto = (
    "O presidente da república discursou na abertura da conferência sobre mudanças climáticas "
    "e afirmou que o Brasil irá cumprir as metas estabelecidas no Acordo de Paris. Ele destacou "
    "a importância da preservação da Amazônia e o compromisso com fontes renováveis de energia."
)


# 🚀 Gerar resumo

In [ ]:
resumo = sumarizador(
    texto,
    max_length=80,
    min_length=20,
    do_sample=False
)

print("Resumo:", resumo[0]['summary_text'])


# ✅ Saída esperada


Resumo: O presidente reafirmou o compromisso do Brasil com o Acordo de Paris e destacou a preservação da Amazônia.

# 📌 Observações
Esse exemplo utiliza inferência direta com modelo pré-treinado (sem fine-tuning).

O modelo ptt5-base foi treinado especificamente para tarefas em português.

Para textos mais longos, ajuste os parâmetros max_length, min_length e ative truncation=True se necessário.



# 🧠 Fine-Tuning de Sumarização com T5 em Português 🇧🇷

Este roteiro mostra como realizar o fine-tuning de um modelo T5 adaptado para a língua portuguesa, utilizando a biblioteca 🤗 Hugging Face Transformers e o dataset XLSum (versão em português).


# 📚 Carregar dataset de exemplo

In [ ]:
from datasets import load_dataset

# Dataset multilíngue com textos e resumos jornalísticos
dataset = load_dataset("csebuetnlp/xlsum", "portuguese")
dataset = dataset.train_test_split(test_size=0.2)
dataset["validation"] = dataset["test"]


# 🧹 Pré-processamento e tokenização

In [ ]:
from transformers import AutoTokenizer

checkpoint = "unicamp-dl/ptt5-base-portuguese-vocab"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

max_input_length = 512
max_target_length = 128

def preprocess_function(examples):
    inputs = ["sumarize: " + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = dataset.map(preprocess_function, batched=True)


# 🧠 Carregar modelo T5 para sumarização

In [ ]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)


# ⚙️ Configuração do treinamento

In [ ]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=False  # Ativar se estiver usando GPU com suporte
)


# 📊 Avaliação com ROUGE

In [ ]:
import evaluate

rouge = evaluate.load("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    return rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)


# 🏋️ Treinar o modelo

In [ ]:
from transformers import Trainer, DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()


# ✅ Avaliação final no conjunto de validação

In [ ]:
trainer.evaluate()


# 📌 Observações
O modelo unicamp-dl/ptt5-base-portuguese-vocab é otimizado para tarefas como sumarização, tradução e geração de texto em português.

O dataset XLSum contém pares (texto, resumo) para dezenas de idiomas, incluindo o português.

O parâmetro predict_with_generate=True permite gerar sequências durante a avaliação, o que é essencial para calcular ROUGE.

